In [1]:
import torch
import numpy as np
import csv
import pandas as pd

import json

from torch_geometric.data import Data
from torch_geometric.loader import NeighborLoader
from torch_geometric.nn import GCNConv

import torch.nn.functional as F
from torch.optim import SGD
from torch import nn

from tqdm import tqdm

from collections import defaultdict

from sklearn.manifold import TSNE

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
from dataset import Dataset
from modules import GCN
from train import train_model
from predict import model_inference

# Load Data

In [4]:
data = Dataset('datasets/facebook.npz', device=device)

In [5]:
dataloader = data.data_loader(batchsize=32)

# Build Model

# Train Model

In [17]:
gcn_model = GCN(data.graph.x.shape[1], 4, hidden_layers=[32]*2).to(device)

optimizer = SGD(gcn_model.parameters(), lr=0.25)

CELoss = torch.nn.CrossEntropyLoss()

train_model(gcn_model, optimizer, CELoss, dataloader)

199/200, Loss 0.4393: 100%|██████████| 200/200 [01:18<00:00,  2.55it/s]


In [18]:
prediction = model_inference(gcn_model, data.graph)

print("> Inference Complete")
accuracy = (prediction == data.graph.y).sum()/len(data.graph.y)
test_accuracy = (prediction[data.graph.test_mask] == data.graph.y[data.graph.test_mask]).sum()/len(data.graph.y[data.graph.test_mask])

print("> Model Results")
print(f"\tTest Accuracy: {test_accuracy}")
print(f"\tFull graph Accuracy: {accuracy}")

> Inference Complete
> Model Results
	Test Accuracy: 0.8457943797111511
	Full graph Accuracy: 0.8460614085197449
